In [31]:
import pandas as pd
import numpy as np
from pandas import DataFrame, Series
import matplotlib.pyplot as plt
from pylab import rcParams
%pylab inline
rcParams['figure.figsize'] = 12, 6
from datetime import datetime
import re
import math
from sklearn.metrics import make_scorer
from sklearn import preprocessing
import xgboost as xgb
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.grid_search import GridSearchCV
from sklearn.cross_validation import cross_val_predict

Populating the interactive namespace from numpy and matplotlib


`%matplotlib` prevents importing * from pylab and numpy


In [32]:
d = pd.read_csv('tmp/engineered_features_2.csv')
del d['Date']
del d['DateYear']
del d['DateDay']
del d['Store']
del d['Id']

In [33]:
d = pd.get_dummies(d, columns=['DateMonth'])

In [34]:
s = d.Sales

In [35]:
for c in d.columns:
    if (
    c.startswith('PromoInterval') or
    (c.startswith('StoreDay') and c != 'StoreDaySales_50') or
    (c.startswith('StoreSal') and c != 'StoreSales_50') or
    c.startswith('Promo2Sin')
    ):
        del d[c]

In [36]:
d = d.fillna(0)
e = DataFrame(preprocessing.scale(d), columns=d.columns)
d = e
d.Sales = s

In [37]:
for c in d.columns:
    d[c] = d[c].astype(float)

In [38]:
tr = d[pd.notnull(d.Sales)]
tr = tr[:int(0.25*len(tr))]
trl = tr.Sales
del tr['Sales']

In [39]:
te = d[pd.isnull(d.Sales)]
tel = te.Sales
del te['Sales']

In [40]:
m = SVR(kernel='rbf', degree=3, C=1.0, epsilon=0.1)
m.fit(tr, trl)

SVR(C=1.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma=0.0,
  kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)

In [41]:
svr_te = m.predict(te)
sub = DataFrame({'Id': np.arange(1, len(svr_te)+1), 'Sales': svr_te})
sub.to_csv('output/s_svr_01.csv', index=False)